Tinder Reviews Sentiment Analysis uygulaması için modelimizi kaydedeceğiz

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("tinder_google_play_reviews.csv")

In [4]:
df.shape

(664865, 11)

In [5]:
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,185e78c8-c8b8-4050-8631-67a4f1e5aff3,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,easy to use and always gets matches i lkke so....,5,0,16.31.0,2026-01-02 00:13:28,NaN,NaN,16.31.0
1,1fe4ed79-7899-4e12-934b-c8b2229ec244,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,keep the good work 👏,5,0,16.35.0,2026-01-01 23:58:37,NaN,NaN,16.35.0
2,335e72a9-7b33-49c2-b739-913a71b7264a,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,worst dating app you guys made me loose a matc...,1,0,16.33.0,2026-01-01 23:11:32,NaN,NaN,16.33.0
3,ddef0666-c6ef-4abf-ba0f-08d0b001c1d1,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,I love this app 💗,5,0,16.27.1,2026-01-01 22:52:28,NaN,NaN,16.27.1
4,3ea6d100-996f-4bfc-ab44-f93d901d0a4f,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,"Well i met someone 4 years ago, getting marrie...",5,0,16.35.0,2026-01-01 22:11:51,NaN,NaN,16.35.0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 664865 entries, 0 to 664864
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   reviewId              664865 non-null  object
 1   userName              664865 non-null  object
 2   userImage             664865 non-null  object
 3   content               663518 non-null  object
 4   score                 664865 non-null  int64 
 5   thumbsUpCount         664865 non-null  int64 
 6   reviewCreatedVersion  529466 non-null  object
 7   at                    664865 non-null  object
 8   replyContent          58696 non-null   object
 9   repliedAt             58696 non-null   object
 10  appVersion            529466 non-null  object
dtypes: int64(2), object(9)
memory usage: 55.8+ MB


In [7]:
df.isnull().sum()

reviewId                     0
userName                     0
userImage                    0
content                   1347
score                        0
thumbsUpCount                0
reviewCreatedVersion    135399
at                           0
replyContent            606169
repliedAt               606169
appVersion              135399
dtype: int64

In [8]:
df = df[['content', 'score']]

In [9]:
df = df.dropna()

In [10]:
df = df[df['content'].str.len() > 10]

In [11]:
df.reset_index(drop=True, inplace=True)

In [12]:
df.shape

(507863, 2)

In [13]:
def rating_to_sentiment(r):
    if r >= 4:
        return "Positive"
    elif r == 3:
        return "Neutral"
    else:
        return "Negative"

In [14]:
df['sentiment'] = df['score'].apply(rating_to_sentiment)

In [15]:
df['sentiment'].value_counts()

sentiment
Negative    303835
Positive    162035
Neutral      41993
Name: count, dtype: int64

In [16]:
import re

In [17]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"[^a-z\s]", "", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

In [18]:
df['clean_text'] = df['content'].apply(clean_text)

In [19]:
X = df['clean_text']

In [20]:
y = df['sentiment']

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [24]:
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1,2))

In [25]:
X_train_tfidf = tfidf.fit_transform(X_train)

In [26]:
X_test_tfidf = tfidf.transform(X_test)

In [27]:
from sklearn.linear_model import LogisticRegression

In [28]:
model = LogisticRegression(max_iter=1000)

In [29]:
model.fit(X_train_tfidf, y_train)

LogisticRegression(max_iter=1000)

In [30]:
model.score(X_train_tfidf, y_train)

0.8392158310566344

In [31]:
y_pred = model.predict(X_test_tfidf)

In [32]:
from sklearn.metrics import accuracy_score, classification_report

In [33]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.8320715150679807
              precision    recall  f1-score   support

    Negative       0.85      0.94      0.89     60767
     Neutral       0.38      0.06      0.10      8399
    Positive       0.81      0.83      0.82     32407

    accuracy                           0.83    101573
   macro avg       0.68      0.61      0.61    101573
weighted avg       0.80      0.83      0.80    101573



In [34]:
import joblib

In [35]:
joblib.dump(model, "sentiment_model.pkl")

['sentiment_model.pkl']

In [36]:
joblib.dump(tfidf, "tfidf_vectorizer.pkl")

['tfidf_vectorizer.pkl']

model.score: 0.84

accuracy_score: 0.83